# Importing libraries

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
pd.set_option("display.max_columns", 100)

from datetime import datetime

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


# for Q-Q plots
import scipy.stats as stats
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Bike sharing
## ITI Alex G1

**Nouhrhan Elaraby**

**Lujain Kotb**

**Ibrahim Yaseen**

**Mohamed Abd Elrazik**

**Sondos Omar**

# Reading the data

In [ ]:
data=pd.read_csv('../input/bike-sharing-demand/train.csv')


In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
col_names=list(data.columns)
for index,col_name in enumerate(col_names):
    print(index ," :" ,col_name)

In [ ]:
data=data.rename(columns={col_names[5]: "Temperature(C)",col_names[6]:"Dew Temperature(C)",col_names[7]: "humidity(%)",
                          col_names[8]:"Wind Speed(m/s)",col_names[11]:"Number of rented bikes"
              })

In [ ]:
data.info()

# Checking Null Values

In [ ]:
data.isna().sum().plot(kind="bar")
plt.show()

In [ ]:
data.isna().sum()

# Checking the type of Data

In [ ]:

data.info()

# Converting Date and splitting it

In [ ]:
def convert_date(df):
    
    
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['month'] = df['datetime'].dt.month
    df['hour'] = df['datetime'].dt.hour
    df['weekday'] = df['datetime'].dt.dayofweek
    df["day"]=df["datetime"].dt.day
    df["year"]=df["datetime"].dt.year
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['month_start'] = df['datetime'].dt.is_month_start
    df['woy'] = df['datetime'].dt.isocalendar().week.astype(int)
    
    return df

In [ ]:
data=convert_date(data)

In [ ]:
data.head()

# Statitical description of the features

In [ ]:
data.describe().T

## Visualizations

## showing distribution of rented bikes

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(25, 20))
sns.boxplot(y=data["Number of rented bikes"],ax=ax[0][0])
ax[0][0].set_title("Box plot on Count ")
sns.boxplot(x="season",y="Number of rented bikes",data=data,ax=ax[0][1])
ax[0][1].set_title("Box plot on Count Across Seasons")
sns.boxplot(x="hour",y="Number of rented bikes",data=data,ax=ax[0][2])
ax[0][2].set_title("Box plot on Count Across hour")
sns.boxplot(x="holiday",y="Number of rented bikes",data=data,ax=ax[1][0])
ax[1][0].set_title("Box plot on Count Across Holiday")
sns.boxplot(x="workingday",y="Number of rented bikes",data=data,ax=ax[1][1])
ax[1][1].set_title("Box plot on Count Across Functioning day")
sns.boxplot(x="dayofweek",y="Number of rented bikes",data=data,ax=ax[1][2])
ax[1][2].set_title("Box plot on Count Across Day of week")

# Correlations between variables

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data.corr("pearson"),
            vmin=-1, vmax=1,
            cmap='coolwarm',
            annot=True, 
            square=True)

#  Sorted correlation of variables to the target

In [ ]:
df1Corr=pd.DataFrame(data.corr().unstack().sort_values(ascending=False)['Number of rented bikes'],columns=['Correlation to the target'])
df1Corr.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

# Plotting variables to the target

In [ ]:
data.plot(x='datetime',y='Number of rented bikes',kind='kde');

# Seasons with most rents¶

In [ ]:
df1VizSeasons=pd.DataFrame(data.groupby('season').sum()['Number of rented bikes'].sort_values(ascending=False))
df1VizSeasons.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

In [ ]:
plt.figure(figsize=(9,8))
data.groupby('season').sum()['Number of rented bikes'].plot.pie();
plt.title("Number of rented bikes share per season");

# Checking rents per hours

In [ ]:
data.groupby('hour').sum()['Number of rented bikes'].plot.bar()

# Rented bikes vs Days of the week

In [ ]:
df1VizDays=pd.DataFrame(data.groupby('dayofweek').sum()['Number of rented bikes'].sort_values(ascending=False))
df1VizDays.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

Days are encoded 0 Mondays and 6 Sundays

# Temperature vs rented bikes

In [ ]:
data.groupby('Temperature(C)').mean()['Number of rented bikes'].plot()

# Visualizing rents with years, months and hours

In [ ]:
#plt.figure(figsize=(14,14))
sns.barplot(data=data, x='year', y=data['Number of rented bikes']);

In [ ]:
sns.barplot(data=data, x='month', y=data['Number of rented bikes'], hue='year');


In [ ]:
plt.figure(figsize=(14,14))
sns.pointplot(data=data, x='hour', y=data['Number of rented bikes'], hue='dayofweek');

In [ ]:
data.info()

# Dropping not important Columns

In [ ]:
def drop_cols(df):
    df=df.drop(['datetime','casual','registered'],axis=1)
    return df

In [ ]:
data=drop_cols(data)

In [ ]:
data.info()

# Check the outliers of Data

In [ ]:
features_plot=['Number of rented bikes',  'Temperature(C)', 'humidity(%)',
       'Wind Speed(m/s)',  'Dew Temperature(C)',
       ]

In [ ]:
data[features_plot].describe()

In [ ]:
def my_plot(feature):
    plt.hist(data[feature])
    plt.title(feature)
    plt.show()
        
for i in features_plot:
    my_plot(i)

In [ ]:
def diagnostic_plots(df, variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins='auto')
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [ ]:
# Average values across each of the categorical columns 
fig = plt.figure(figsize=(15, 12))
axes = fig.add_subplot(2, 2, 1)
group_season = pd.DataFrame(data.groupby(['season'])['Number of rented bikes'].mean()).reset_index()
sns.barplot(data=group_season, x='season', y='Number of rented bikes', ax=axes)
axes.set(xlabel='season', ylabel='Number of rented bikes', title='Average bike rentals across Seasons')

axes = fig.add_subplot(2, 2, 2)
group_workingday = pd.DataFrame(data.groupby(['workingday'])['Number of rented bikes'].mean()).reset_index()
sns.barplot(data=group_workingday, x='workingday', y='Number of rented bikes', ax=axes)
axes.set(xlabel='workingday', ylabel='Number of rented bikes', title='Average bike rentals across Working Day')

axes = fig.add_subplot(2, 2, 3)
group_season = pd.DataFrame(data.groupby(['holiday'])['Number of rented bikes'].mean()).reset_index()
sns.barplot(data=group_season, x='holiday', y='Number of rented bikes', ax=axes)
axes.set(xlabel='holiday', ylabel='Number of rented bikes', title='Average bike rentals across Holiday')
plt.show()

# Hourly count based on functioning day

In [ ]:
# seaborn boxplots across hours
f, axes = plt.subplots(1, 1, figsize=(15, 6))
sns.boxplot(data=data, y='Number of rented bikes', x='hour', hue='workingday', ax=axes)
handles, _ = axes.get_legend_handles_labels()
axes.legend(handles, ['Not a workingday', 'workingday'])
axes.set(title='Hourly Count based on Functioning day or not')

plt.show()

# Plotting average bike count for each hour as a function of various categories.

In [ ]:

# Plots of average count across hour in a day for various categories

f, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 18))
group_work_hour = pd.DataFrame(data.groupby(['workingday', 'hour'])['Number of rented bikes'].mean()).reset_index()
sns.pointplot(data=group_work_hour, x='hour', y='Number of rented bikes', hue='workingday', ax=axes[0], legend=True)
handles, _ = axes[0].get_legend_handles_labels()
axes[0].legend(handles, ['Not a workingday', 'workingday'])
axes[0].set(xlabel='Hour in the day', ylabel='workingday', title='Average Bike Rentals by the day if Working day or Not')

hue_order= [0,1,2,3,4,5,6]
group_day_hour = pd.DataFrame(data.groupby(['day', 'hour'])['Number of rented bikes'].mean()).reset_index()
sns.pointplot(data=group_day_hour, x='hour', y='Number of rented bikes', hue='day', ax=axes[1], hue_order=hue_order)
axes[1].set(xlabel='Hour in the day', ylabel='Number of rented bikes', title='Average Bike Rentals by the day across Weekdays')


plt.show()

# Monthly distributions

In [ ]:
# Average Monthly Count Distribution plot
f, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
group_month = pd.DataFrame(data.groupby(['month', 'workingday'])['Number of rented bikes'].mean()).reset_index()
sns.barplot(data=group_month, x='month', y='Number of rented bikes', hue='workingday', ax=axes)
axes.set(xlabel='Month', ylabel='Number of rented bikes', title='Average bike rentals per Month')
handles, _ = axes.get_legend_handles_labels()
axes.legend(handles, ['workingday', 'workingday'])
plt.show()

# Rented bikes vs Temperature, Humidity and Windspeed

In [ ]:
# Regression Plots with respect to Temperature, Humidity and Windspeed
fig = plt.figure(figsize=(18, 8))
axes = fig.add_subplot(1, 3, 1)
sns.regplot(data=data, x='Temperature(C)', y='Number of rented bikes',ax=axes)
axes.set(title='Reg Plot for Temperature vs. Number of rented bikes')
axes = fig.add_subplot(1, 3, 2)
sns.regplot(data=data, x='humidity(%)', y='Number of rented bikes',ax=axes, color='r')
axes.set(title='Reg Plot for Humidity vs. Number of rented bikes')
axes = fig.add_subplot(1, 3, 3)
sns.regplot(data=data, x='Wind Speed(m/s)', y='Number of rented bikes',ax=axes, color='g')
axes.set(title='Reg Plot for Windspeed vs. Number of rented bikes')
plt.show()

# Showing the distribution of categorical variables

# Number of rented bikes

In [ ]:
diagnostic_plots(data, 'Number of rented bikes')

# Temperatures

In [ ]:
#showing distribution for temperature
diagnostic_plots(data, 'Temperature(C)')

# Humidity

In [ ]:
diagnostic_plots(data, 'humidity(%)')

# Dew Temperature

In [ ]:
diagnostic_plots(data, 'Dew Temperature(C)')

In [ ]:
data.shape   # the data size is much larger than number of outliers in data

## Handling Outliers

In [ ]:
def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

# showing correlation between variables

In [ ]:
plt.figure(figsize=(20,20))
correlation=data.corr()
sns.heatmap(correlation,annot=True)

In [ ]:
# data.drop("Dew Temperature(C)",axis=1,inplace=True)

In [ ]:
data.columns

# applying additional function to data :peak,ideal,sticky

In [ ]:
data.info()

In [ ]:
def additional(df):
    df['ideal'] = df[['Temperature(C)', 'Wind Speed(m/s)']].apply(lambda x: (0, 1)[x['Temperature(C)'] > 27 and x['Wind Speed(m/s)'] < 30], axis = 1)
    df['sticky'] = df[['humidity(%)', 'workingday']].apply(lambda x: (0, 1)[x['workingday'] == 1 and x['humidity(%)'] >= 60], axis = 1)
    df["windchill"]=13.12+0.6215*df['Temperature(C)']-11.37*(df['Wind Speed(m/s)']*3.6)**0.16+0.3965*df['Temperature(C)'] \
    *(df['Wind Speed(m/s)']*3.6)**0.16
    df['newweather']=17.3+df['Temperature(C)']-0.11*df['humidity(%)']+0.34*df['Wind Speed(m/s)']
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/23.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/23.0)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['log_wind']=np.log1p(df['Wind Speed(m/s)'])
    df['log_hum']=np.log1p(df['humidity(%)'])
    
    return df

In [ ]:
cond1 = data['workingday']== 'Yes'
cond2 = data['hour'].isin([7,8,9,17,18,19])
cond3 = data['hour'].isin([10,11,12,13,14,15,16])
cond4 = data['hour'].isin([0,1,2,3,4,5,6,20,21,22,23])

conditions =[~cond1,cond1&cond2,cond1&cond3,cond1&cond4]
vals = ['No','High','Medium','Low']
data['Map demand'] = np.select(conditions,vals)


In [ ]:
data=additional(data)

In [ ]:
data.head()

# Showing the distribution of categorical variables

In [ ]:
cat_features = data.columns[data.dtypes=='object']
cat_features=list(cat_features)
cat_features

In [ ]:
sns.countplot(x=cat_features[0],data=data)
plt.show()

In [ ]:
def count_category(name):
    print(name)
    print(data[name].value_counts())
    print("-------------------------------------------------")
for i in cat_features:
    count_category(i)

# EDA

In [ ]:
count_bikes_by_hour = data.groupby("hour")["Number of rented bikes"].sum()
print(count_bikes_by_hour)

In [ ]:
count_bikes_by_hour.plot(kind="bar",
title="count bikes by hour")
plt.savefig('count bikes by hr')
plt.show()

In [ ]:
count_bikes_by_Holiday = data.groupby("holiday")["Number of rented bikes"].sum()
print(count_bikes_by_Holiday)

In [ ]:
data.groupby("holiday")["Number of rented bikes"].sum().plot(kind="bar")
plt.savefig("count bikes by holiday")
plt.show()

In [ ]:
count_bikes_by_funcday = data.groupby("workingday")["Number of rented bikes"].sum()
print(count_bikes_by_funcday)

In [ ]:
data.groupby("workingday")["Number of rented bikes"].sum().plot(kind="bar")
plt.savefig("count_bikes_by_funcday")
plt.show()

In [ ]:
count_bikes_by_Season = data.groupby("season")["Number of rented bikes"].sum()
print(count_bikes_by_Season)

In [ ]:
count_bikes_by_Season.plot(kind="bar")

## conclusion

There is a peak for renting bikes at 8 am and 6 pm which is most probably the time a person is going to or leaving his work

This is the order of season for number of rented bikes(descending order) ,

1)Autumn

2)Summer

3)Spring,

4)Winter

Most people rent bikes on Functioning days and days which aren't holidays

# Rolling Mean with window of 3

In [ ]:
def rolling_mean(df,cols,window=3):
    for col in cols:
        new_col = col+'_rolled'
        df[new_col] = df[col].rolling(window=window).mean()
    return df


In [ ]:
data =  rolling_mean(data,['Temperature(C)','Wind Speed(m/s)',
                        'humidity(%)','weather'])


In [ ]:
def shifting(df,cols):
    for col in cols:
        new_col = col+'_shifted'
        df[new_col] = df[col].shift(periods=-1)
    return df
data =  shifting(data,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])
def shifting_1(df,cols):
    for col in cols:
        new_col = col+'_shifted1'
        df[new_col] = df[col].shift(periods=-2)
    return df
data =  shifting_1(data,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])

def shifting_2(df,cols):
    for col in cols:
        new_col = col+'_shifted2'
        df[new_col] = df[col].shift(periods=-3)
    return df
data =  shifting_2(data,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])

In [ ]:
data=data.fillna(data.mean())

In [ ]:
data.head()

# Converting Categorical Varibales

In [ ]:
def conv_cat_features(df):
    df['holiday'].replace({"No Holiday":1, "Holiday":0}, inplace=True)
    df['workingday'].replace({"Yes":1, "No":0}, inplace=True)
    data_cat_transformed = pd.get_dummies(df, drop_first=True)
    return data_cat_transformed

In [ ]:
data_cat_transformed=conv_cat_features(data)
# Print the number of features after one-hot encoding
print("{} total features after one-hot encoding.".format(len(data_cat_transformed.columns)))
print(data_cat_transformed.columns)#

# Splitting the data into features and Target Label

In [ ]:
# Split the data into features and target label
X = data_cat_transformed.drop('Number of rented bikes', axis=1)
y = data_cat_transformed['Number of rented bikes']
def custom_train_valid_split(data, cutoff_day=15):
    train = data[data['day'] <= cutoff_day]
    valid = data[data['day'] > cutoff_day]
    return train,valid

train,valid =  custom_train_valid_split(data, cutoff_day=15)
train_idx = train.index
valid_idx = valid.index
myCViterator = []
myCViterator.append( (train_idx, valid_idx) )

# Create Training Model

## Define Loss Function

In [ ]:
def RMSLE(y_train,y_pred):
    return np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
def train_predict(regressor): 
    '''
    inputs:
       - regressor: the learning algorithm to be trained and predicted on
    '''
    
    results = {}
    
    # Fit the classifier to the training data and Calculate the time
    start_train = time()
    regressor = regressor.fit(X, y)
    end_train = time()
    results['train_time'] = round((end_train-start_train), 2)

    # Predict the training/testing labels and Calculate the time
    start_test = time()
    y_pred_train = regressor.predict(X)
    y_pred_test = regressor.predict(test_cat_transformed)
    end_test = time()   
    results['pred_time'] = round((end_test-start_test), 2)
    results['RMSLE_train'] =round(RMSLE(y, y_pred_train),4) 
    results['Rsquared_train'] = r2_score(y, y_pred_train)
    results['MSE_train'] = mean_squared_error(y, y_pred_train)
    return results,y_pred_test

# Test Data

In [ ]:
test_df=pd.read_csv('../input/bike-sharing-demand/test.csv')
test_df_copy=test_df.copy()

In [ ]:
test_df.describe()

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.columns

In [ ]:
test_df=test_df.rename(columns={col_names[5]: "Temperature(C)",col_names[6]:"Dew Temperature(C)",col_names[7]: "humidity(%)",
                          col_names[8]:"Wind Speed(m/s)"
              })

In [ ]:
test_df=convert_date(test_df)

In [ ]:
test_df.info()

In [ ]:
cond1 = test_df['workingday']=='Yes'
cond2 = test_df['hour'].isin([7,8,9,17,18,19])
cond3 = test_df['hour'].isin([10,11,12,13,14,15,16])
cond4 = test_df['hour'].isin([0,1,2,3,4,5,6,20,21,22,23])
conditions =[~cond1,cond1&cond2,cond1&cond3,cond1&cond4]
vals = ['No','High','Medium','Low']
test_df['Map demand'] = np.select(conditions,vals)

In [ ]:
test_df=test_df.drop(['datetime',],axis=1)


In [ ]:
test_df=additional(test_df)

In [ ]:
test_df =  rolling_mean(test_df,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])
test_df =  shifting(test_df,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])
test_df =  shifting_1(test_df,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])
test_df =  shifting_2(test_df,['Temperature(C)','Wind Speed(m/s)','weather', 'humidity(%)'])
test_df=test_df.fillna(test_df.mean())

In [ ]:
test_cat_transformed=conv_cat_features(test_df)

# Model Tuning

In [ ]:
# parameters={
#  "learning_rate"    :np.arange(0.04,0.2,0.01)   ,                             # [0.05,0.06,0.065,0.07,0.075,0.08,0.085,0.09,0.10,0.125,0.15,0.2] ,
#   "max_depth"        :np.arange(4,16,1) ,                    #[4,5,6,7,8,9,10]
#   "min_child_weight" : np.arange(11,17, 1),  #[1,2,3,4,5,6,7,8,9,10],
#   "gamma"            : np.arange(2,10,0.5)  ,               #    [0.5,1,2,3,5,7,6,8,9,10],
#   "n_estimators"     :np.arange(2500,3500, 50),      # [500,700,800,900,1000,1200,1300,1400,1500,1600,1700,1800,1900,2000,2200,2500,2750],
#   "subsample"        :np.arange(0.7, 1, 0.02) ,
#   "colsample_bytree" :np.arange(0.4,1,0.05), # [0.7,0.75,0.8,0.85,0.9,0.95,1],
#   "tree_method"    : ["gpu_hist"]
#      }

In [ ]:
# def model_tuning(reg, parameters):    

#     # Evaluate using neg_mean_squared_log_error 
#     #scorer = make_scorer(RMSLE,greater_is_better=False)

#     # Perform grid search on the regressor and tune parameters, using scorer evaluation
#     grid_obj = RandomizedSearchCV(estimator=reg, param_distributions=parameters,n_iter=300, 
#                                   scoring="neg_mean_squared_log_error",n_jobs=-1,cv=5,verbose=3)

#     # Fit the grid search object to the training data and find the optimal parameters
#     grid_fit = grid_obj.fit(X, y)

#     # Get the best estimator
#     best_clf = grid_fit.best_estimator_
#     print(f"best mean cross validation score: {grid_fit.best_score_}")
#     print(f"best parameters: {grid_fit.best_params_}")
    
#     # Make predictions using the optimized and the best model
#     y_best_pred = best_clf.predict(test_cat_transformed)
    
#     return best_clf,grid_fit, y_best_pred

In [ ]:
# clf, best_clf,grid_fit, y_best_pred=model_tuning(regressor, parameters)

In [ ]:
# best_clf

In [ ]:
# grid_fit

> After Running HyperParameter Tuning jobs on xgboost model and catboost model the optimum parameters reached were:
> - XGBoost: {}
> - CatBoost: {}

In [ ]:
X_train=X.copy()
X_test=test_cat_transformed.copy()

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

In [ ]:
# X_train_xgb=X_train.drop(['hour_sin', 'hour_cos','month_sin', 'month_cos','woy','windchill', 
#                       'newweather','Temperature(C)_rolled','Dew Temperature(C)',
#                       'humidity(%)','Wind Speed(m/s)_rolled', 
#                       ],axis=1)
# X_test_xgb=X_test.drop(['hour_sin', 'hour_cos','month_sin', 'month_cos','woy','windchill', 
#                     'newweather','Temperature(C)_rolled','Dew Temperature(C)',
#                     'humidity(%)','Wind Speed(m/s)_rolled',
#                     ],axis=1)

In [ ]:
xgb_v = XGBRegressor(
   objective='count:poisson',random_state=42)



xgb= XGBRegressor(
    objective='count:poisson',random_state=42,
   subsample= 0.83, n_estimators= 3000, 
   min_child_weight= 16, max_depth= 5, 
   learning_rate= 0.055, gamma= 1, colsample_bytree= 0.5)

catboost_v=CatBoostRegressor(objective='Poisson',iterations=4000,l2_leaf_reg=1,learning_rate=0.054  , silent = True)
catboosttrial=CatBoostRegressor(objective='Poisson',iterations=4000,l2_leaf_reg=1,learning_rate=0.054,silent=True)                              
xgb_models={
    "XGB_vanella":xgb_v,
    "XGB_mod" : xgb }
cat_models ={
    "Catbbost_mod" :catboosttrial,
    "Catboost_Vanilla" :catboost_v}

In [ ]:
columns = ['Classifier', 'RMSLE']

df_scores_cat = pd.DataFrame(columns=columns)
for name, model in cat_models.items():
    model.fit(X_train_cat, y)
    Y_predicted = model.predict(X_train_cat)
    rmsle = np.sqrt(mean_squared_log_error(y, Y_predicted))
    df_row = pd.DataFrame([[name, rmsle]], columns=columns)
    df_scores_cat = df_scores_cat.append(df_row, ignore_index=True)

In [ ]:
columns = ['Classifier', 'RMSLE']

df_scores_xgb = pd.DataFrame(columns=columns)
for name, model in xgb_models.items():
    model.fit(X_train, y)
    Y_predicted = model.predict(X_train)
    rmsle = np.sqrt(mean_squared_log_error(y, Y_predicted))
    df_row = pd.DataFrame([[name, rmsle]], columns=columns)
    df_scores_xgb = df_scores_xgb.append(df_row, ignore_index=True)

In [ ]:
df_scores_cat.sort_values(by=['RMSLE'])

In [ ]:
df_scores_xgb.sort_values(by=['RMSLE'])

# Plot Feature importance from Xgboostregressor without any hyperparameter tuning

In [ ]:
from xgboost import plot_importance, plot_tree
fig = plt.figure(figsize=(15,12))
fig = plot_importance(xgb_v, height=0.9,max_num_features = 25)
plt.show()

# Plot Feature importance from Xgboostregressor with best parameters after hyperparameter tuning

In [ ]:
fig = plt.figure(figsize=(15,12))
fig = plot_importance(xgb, height=0.9,max_num_features = 25)
plt.show()

# Getting the results on Test Data

In [ ]:
y_cat_pred=catboosttrial.predict(X_test_cat)

In [ ]:
y_xgb_pred=xgb.predict(X_test)

In [ ]:
# # #getting the output from train predicgt func
# test_df_copy['yxgb']=y_xgb_pred.astype('int')
# test_df_copy['ycat']=y_cat_pred.astype('int')

In [ ]:
# yavg = 0.64*test_df_copy['yxgb'] + 0.36*test_df_copy['ycat']


In [ ]:
test_df_copy['count'] = y_cat_pred.astype('int')

In [ ]:
test_df_copy[test_df_copy["workingday"]=='No']

In [ ]:
test_df_copy[['datetime','count']].to_csv("submissions1.csv",index=False)